In [1]:
import numpy as np
import keras2onnx
import onnxruntime
from keras.models import Sequential, load_model, Model
from keras.datasets import mnist
from metrics import *
import tensorflow.compat.v1 as tf
from glob import glob
import os
import facenet
from keras.preprocessing import image

Using TensorFlow backend.
Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!
W0424 22:09:41.696109 140470669856128 wrapper.py:21] Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!
W0424 22:09:41.800404 140470669856128 deprecation.py:323] From /home/aswin/anaconda3/envs/enscalo_test/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
(X, y), (X_test, y_test) = mnist.load_data()
X = X[:, :, :, np.newaxis].astype('float32') / 255
X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255

In [21]:
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(160, 160))
    return np.expand_dims(np.asarray(img),axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [22]:
classes, nrof_classes = facenet.get_dataset("vggface2_train_2/train")
path_exp = os.path.expanduser("vggface2_train_2/train")
image_paths = [glob(os.path.join(path_exp, classes[i], "*.jpg")) for i in range(nrof_classes)]
split = int(round(nrof_classes*(1-0.99)))
X_test = paths_to_tensor([image_paths[0][0],image_paths[0][1]])

In [23]:
arcface_model = load_model('models/mnist_vgg8_arcface_5d/model.hdf5', custom_objects={'ArcFace': ArcFace})
arcface_model = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-3].output)
arcface_features = arcface_model.predict(X_test, verbose=1)
arcface_features /= np.linalg.norm(arcface_features, axis=1, keepdims=True)

[(None, 5), (None, 86)]
2/2 [==============================] - 0s 155ms/step


In [26]:
image_paths[0][0].split("/")

['vggface2_train_2', 'train', 'n000002', '0161_01.jpg']

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig2 = plt.figure()
ax2 = Axes3D(fig2)
for c in range(len(np.unique(y_test))):
    ax2.plot(arcface_features[y_test==c, 0], arcface_features[y_test==c, 1], arcface_features[y_test==c, 2], '.', alpha=0.1)
plt.title('ArcFace')

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_generator = datagen.flow_from_directory(
        'data/train',
        target_size=(1580, 862),
        batch_size=32)
validation_generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(1580, 862),
        batch_size=32,
        class_mode='binary')


In [1]:
import imagehash, cv2
from PIL import Image

In [ ]:
imagehash.average_hash(
    Image.open("vggface2_test/test/n000029/0130_01.jpg")
)

In [2]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
img = Image.open("vggface2_test_2/test/n000029/0130_01.jpg").convert('L')
face_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml')
faces_multi = face_cascade.detectMultiScale(np.asarray(img), 1.3, 5)